# Requirements

In [ ]:
%pip install -r requirements.txt

# install torch and torchvision (select the proper cuda version to suit your machine)
%pip install pytorch torchvision cudatoolkit=9.0 -c pytorch

# install torchreid (don't need to re-build it if you modify the source code)


In [4]:
!python setup.py develop

Compiling torchreid/metrics/rank_cylib/rank_cy.pyx because it changed.
[1/1] Cythonizing torchreid/metrics/rank_cylib/rank_cy.pyx
running develop
running egg_info
creating torchreid.egg-info
writing torchreid.egg-info\PKG-INFO
writing dependency_links to torchreid.egg-info\dependency_links.txt
writing requirements to torchreid.egg-info\requires.txt
writing top-level names to torchreid.egg-info\top_level.txt
writing manifest file 'torchreid.egg-info\SOURCES.txt'
dependency c:\Users\Admin\anaconda3\envs\endocv_311\Lib\site-packages\numpy\core\include\numpy\arrayobject.h won't be automatically included in the manifest: the path must be relative
dependency c:\Users\Admin\anaconda3\envs\endocv_311\Lib\site-packages\numpy\core\include\numpy\arrayscalars.h won't be automatically included in the manifest: the path must be relative
dependency c:\Users\Admin\anaconda3\envs\endocv_311\Lib\site-packages\numpy\core\include\numpy\ndarrayobject.h won't be automatically included in the manifest: the p

d:\DANC\deep-person-reid\torchreid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
c:\Users\Admin\anaconda3\envs\endocv_311\Lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: d:\DANC\deep-person-reid\torchreid\metrics\rank_cylib\rank_cy.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
c:\Users\Admin\anaconda3\envs\endocv_311\Lib\site-packages\setuptools\command\develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
     

In [5]:
!mkdir reid-data

# Main

In [1]:
import torchreid

d:\DANC\deep-person-reid\torchreid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [2]:
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources="endocv",
    targets="endocv",
    height=128,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

Building train transforms ...
+ resize to 128x128
+ random flip
+ random crop (enlarge to 144x144 and crop 128x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 128x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded EndoCV
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    51 |     1454 |         8
  query    |    51 |      262 |         8
  gallery  |    51 |      507 |         1
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded EndoCV
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    51 |     1454 |         8
  query    |    51 |      262 |         8
  gallery  | 

d:\DANC\deep-person-reid\torchreid\data\datasets\image\endocv.py:34: UserWarning: The current data structure is deprecated. Please put data folders such as "bounding_box_train" under "Market-1501-v15.09.15".
  warnings.warn(


In [3]:
model = torchreid.models.build_model(
    name="osnet_dcn_x0_25_endocv", #osnet_x0_25
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

Successfully loaded imagenet pretrained weights from "C:\Users\Admin/.cache\torch\checkpoints\osnet_x0_25_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['conv5.conv.weight', 'classifier.weight', 'classifier.bias']


In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)


In [6]:
engine.run(
    save_dir="log/osnet_dcn_x0_25_endocv",
    max_epoch=100,
    eval_freq=10,
    print_freq=10
    # if validate, enable 3 lines below
    # test_only=True,
    # visrank=True,
    # visrank_topk=5

)

=> Start training
epoch: [1/100][10/45]	time 0.115 (2.414)	data 0.000 (2.230)	eta 3:00:36	loss 3.6228 (3.8075)	acc 18.7500 (10.6250)	lr 0.000300
epoch: [1/100][20/45]	time 0.109 (1.267)	data 0.000 (1.115)	eta 1:34:35	loss 3.1680 (3.6517)	acc 25.0000 (14.5312)	lr 0.000300
epoch: [1/100][30/45]	time 0.119 (0.886)	data 0.001 (0.744)	eta 1:06:00	loss 3.2916 (3.5523)	acc 15.6250 (15.4167)	lr 0.000300
epoch: [1/100][40/45]	time 0.082 (0.693)	data 0.000 (0.558)	eta 0:51:29	loss 3.0721 (3.4583)	acc 31.2500 (18.5156)	lr 0.000300
epoch: [2/100][10/45]	time 0.101 (2.315)	data 0.000 (2.200)	eta 2:51:30	loss 2.9540 (2.8142)	acc 31.2500 (36.2500)	lr 0.000300
epoch: [2/100][20/45]	time 0.105 (1.210)	data 0.000 (1.100)	eta 1:29:26	loss 2.9454 (2.8202)	acc 28.1250 (35.6250)	lr 0.000300
epoch: [2/100][30/45]	time 0.109 (0.845)	data 0.001 (0.734)	eta 1:02:21	loss 2.5018 (2.7769)	acc 53.1250 (36.3542)	lr 0.000300
epoch: [2/100][40/45]	time 0.091 (0.661)	data 0.000 (0.550)	eta 0:48:40	loss 2.3495 (2.7328)	

In [7]:
%pip install numpy==1.24.4

  Using cached numpy-1.24.4-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-win_amd64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.


In [ ]:
!python tools/visualize_actmap.py \
--root /content/deep-person-reid/reid-data \
-d endocv \
-m osnet_dcn_x0_5_endocv \
--weights /content/drive/MyDrive/log/osnet_dcn_x0_5_endocv/model/osnet_dcn_x0_5_endocv_80.pt \
--save-dir log/visactmap_osnet_dcn_x0_5_endocv

2025-04-12 09:14:06.015013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744449246.036297    8326 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744449246.042681    8326 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
/content/deep-person-reid/torchreid/data/datasets/image/endocv.py:34: UserWarning: The curre

In [ ]:
import torch

model = torch.load('/content/drive/MyDrive/log/osnet_dcn_x0_5_endocv/model/model.pth.tar-100',weights_only=True)


# Lưu mô hình vào tệp pt
torch.save(model, '/content/drive/MyDrive/log/osnet_dcn_x0_5_endocv/model/osnet_dcn_x0_5_endocv_80.pt')

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy._core.multiarray.scalar was not an allowed global by default. Please use `torch.serialization.add_safe_globals([scalar])` or the `torch.serialization.safe_globals([scalar])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weig

In [ ]:
from torchreid.utils import FeatureExtractor

extractor = FeatureExtractor(
    model_name='osnet_x0_25',
    model_path='/content/deep-person-reid/osnet_x0_25_model_endo.pt',
    device='cuda'
)

image_list = [
    '/content/deep-person-reid/reid-data/endocv/query/0001_c1s2_000226_01.jpg',
    '/content/deep-person-reid/reid-data/endocv/query/0002_c2s1_000205_01.jpg',
    '/content/deep-person-reid/reid-data/endocv/query/0003_c3s1_000125_01.jpg',
    '/content/deep-person-reid/reid-data/endocv/query/0004_c4s1_000137_01.jpg',
    '/content/deep-person-reid/reid-data/endocv/query/0005_c5s2_000039_01.jpg',
    '/content/deep-person-reid/reid-data/endocv/query/0006_c6s2_000014_01.jpg'
]

features = extractor(image_list)
print(features.shape) # output (5, 512)

Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "/content/deep-person-reid/log/osnetx025/model/model.pth.tar-50"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
torch.Size([6, 512])


In [ ]:
print(features)

tensor([[1.2108, 0.0000, 0.5007,  ..., 0.0000, 0.3413, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.2481, 0.2577, 1.2484],
        [3.1901, 0.0000, 0.9175,  ..., 0.0000, 0.2050, 0.0000],
        [0.0000, 3.2798, 0.0000,  ..., 0.0000, 0.7325, 2.1805],
        [0.0000, 0.7419, 0.0000,  ..., 1.4197, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.7335,  ..., 0.0000, 0.5269, 0.0000]],
       device='cuda:0')


In [ ]:
!cp -r /content/deep-person-reid/log/osnetx025_2/visrank_endocv /content/drive/MyDrive/logs

In [2]:
%pip install torchsummary

  Using cached torchsummary-1.5.1-py3-none-any.whl.metadata (296 bytes)
Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from __future__ import division, absolute_import
import warnings
import torch
from torch import nn
from torch.nn import functional as F
from torchvision.ops import DeformConv2d

__all__ = [
    'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_x0_25_endocv', 'osnet_dcn_x0_5_endocv'
]

pretrained_urls = {
    'osnet_x1_0':
    'https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY',
    'osnet_x0_75':
    'https://drive.google.com/uc?id=1uwA9fElHOk3ZogwbeY5GkLI6QPTX70Hq',
    'osnet_x0_5':
    'https://drive.google.com/uc?id=16DGLbZukvVYgINws8u8deSaOqjybZ83i',
    'osnet_x0_25':
    'https://drive.google.com/uc?id=1rb8UN5ZzPKRc_xvtHlyDh-cSz88YX9hs',
    'osnet_ibn_x1_0':
    'https://drive.google.com/uc?id=1sr90V6irlYYDd4_4ISU2iruoRG8J__6l',
    'osnet_x0_25_endocv':
    'https://drive.google.com/uc?id=1W8mz6skAUmg33zMVpWn6woJym7xGygjh',
    'osnet_dcn_x0_5_endocv':
    'https://drive.google.com/uc?id=16DGLbZukvVYgINws8u8deSaOqjybZ83i'
}


##########
# Basic layers
##########
class ConvLayer(nn.Module):
    """Convolution layer (conv + bn + relu)."""

    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride=1,
        padding=0,
        groups=1,
        IN=False
    ):
        super(ConvLayer, self).__init__()
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=padding,
            bias=False,
            groups=groups
        )
        if IN:
            self.bn = nn.InstanceNorm2d(out_channels, affine=True)
        else:
            self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


class Conv1x1(nn.Module):
    """1x1 convolution + bn + relu."""

    def __init__(self, in_channels, out_channels, stride=1, groups=1):
        super(Conv1x1, self).__init__()
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            1,
            stride=stride,
            padding=0,
            bias=False,
            groups=groups
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


class Conv1x1Linear(nn.Module):
    """1x1 convolution + bn (w/o non-linearity)."""

    def __init__(self, in_channels, out_channels, stride=1):
        super(Conv1x1Linear, self).__init__()
        self.conv = nn.Conv2d(
            in_channels, out_channels, 1, stride=stride, padding=0, bias=False
        )
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x


class Conv3x3(nn.Module):
    """3x3 convolution + bn + relu."""

    def __init__(self, in_channels, out_channels, stride=1, groups=1):
        super(Conv3x3, self).__init__()
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            3,
            stride=stride,
            padding=1,
            bias=False,
            groups=groups
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


class LightConv3x3(nn.Module):
    """Lightweight 3x3 convolution.

    1x1 (linear) + dw 3x3 (nonlinear).
    """

    def __init__(self, in_channels, out_channels):
        super(LightConv3x3, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels, out_channels, 1, stride=1, padding=0, bias=False
        )
        self.conv2 = nn.Conv2d(
            out_channels,
            out_channels,
            3,
            stride=1,
            padding=1,
            bias=False,
            groups=out_channels
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


##########
# Building blocks for omni-scale feature learning
##########
class ChannelGate(nn.Module):
    """A mini-network that generates channel-wise gates conditioned on input tensor."""

    def __init__(
        self,
        in_channels,
        num_gates=None,
        return_gates=False,
        gate_activation='sigmoid',
        reduction=16,
        layer_norm=False
    ):
        super(ChannelGate, self).__init__()
        if num_gates is None:
            num_gates = in_channels
        self.return_gates = return_gates
        self.global_avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(
            in_channels,
            in_channels // reduction,
            kernel_size=1,
            bias=True,
            padding=0
        )
        self.norm1 = None
        if layer_norm:
            self.norm1 = nn.LayerNorm((in_channels // reduction, 1, 1))
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(
            in_channels // reduction,
            num_gates,
            kernel_size=1,
            bias=True,
            padding=0
        )
        if gate_activation == 'sigmoid':
            self.gate_activation = nn.Sigmoid()
        elif gate_activation == 'relu':
            self.gate_activation = nn.ReLU(inplace=True)
        elif gate_activation == 'linear':
            self.gate_activation = None
        else:
            raise RuntimeError(
                "Unknown gate activation: {}".format(gate_activation)
            )

    def forward(self, x):
        input = x
        x = self.global_avgpool(x)
        x = self.fc1(x)
        if self.norm1 is not None:
            x = self.norm1(x)
        x = self.relu(x)
        x = self.fc2(x)
        if self.gate_activation is not None:
            x = self.gate_activation(x)
        if self.return_gates:
            return x
        return input * x


class OSBlock(nn.Module):
    """Omni-scale feature learning block."""

    def __init__(
        self,
        in_channels,
        out_channels,
        IN=False,
        bottleneck_reduction=4,
        **kwargs
    ):
        super(OSBlock, self).__init__()
        mid_channels = out_channels // bottleneck_reduction
        self.conv1 = Conv1x1(in_channels, mid_channels)
        self.conv2a = LightConv3x3(mid_channels, mid_channels)
        self.conv2b = nn.Sequential(
            LightConv3x3(mid_channels, mid_channels),
            LightConv3x3(mid_channels, mid_channels),
        )
        self.conv2c = nn.Sequential(
            LightConv3x3(mid_channels, mid_channels),
            LightConv3x3(mid_channels, mid_channels),
            LightConv3x3(mid_channels, mid_channels),
        )
        self.conv2d = nn.Sequential(
            LightConv3x3(mid_channels, mid_channels),
            LightConv3x3(mid_channels, mid_channels),
            LightConv3x3(mid_channels, mid_channels),
            LightConv3x3(mid_channels, mid_channels),
        )
        self.gate = ChannelGate(mid_channels)
        self.conv3 = Conv1x1Linear(mid_channels, out_channels)
        self.downsample = None
        if in_channels != out_channels:
            self.downsample = Conv1x1Linear(in_channels, out_channels)
        self.IN = None
        if IN:
            self.IN = nn.InstanceNorm2d(out_channels, affine=True)

    def forward(self, x):
        identity = x
        x1 = self.conv1(x)
        x2a = self.conv2a(x1)
        x2b = self.conv2b(x1)
        x2c = self.conv2c(x1)
        x2d = self.conv2d(x1)
        x2 = self.gate(x2a) + self.gate(x2b) + self.gate(x2c) + self.gate(x2d)
        x3 = self.conv3(x2)
        if self.downsample is not None:
            identity = self.downsample(identity)
        out = x3 + identity
        if self.IN is not None:
            out = self.IN(out)
        return F.relu(out)


class DeformableOSBlock(nn.Module):
    """Deformable convolution block for OSNet conv5."""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(DeformableOSBlock, self).__init__()
        self.offset_conv = nn.Conv2d(
            in_channels, 2 * kernel_size * kernel_size, 
            kernel_size=kernel_size, stride=stride, padding=padding
        )
        self.deform_conv = DeformConv2d(
            in_channels, out_channels, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        offset = self.offset_conv(x)
        x = self.deform_conv(x, offset)
        x = self.bn(x)
        x = self.relu(x)
        return x

##########
# Network architecture
##########
class OSNet(nn.Module):
    """Omni-Scale Network.
    
    Reference:
        - Zhou et al. Omni-Scale Feature Learning for Person Re-Identification. ICCV, 2019.
        - Zhou et al. Learning Generalisable Omni-Scale Representations
          for Person Re-Identification. TPAMI, 2021.
    """

    def __init__(
        self,
        num_classes,
        blocks,
        layers,
        channels,
        feature_dim=512,
        loss='softmax',
        IN=False,
        **kwargs
    ):
        super(OSNet, self).__init__()
        num_blocks = len(blocks)
        assert num_blocks == len(layers)
        assert num_blocks == len(channels) - 1
        self.loss = loss
        self.feature_dim = feature_dim

        # convolutional backbone
        self.conv1 = ConvLayer(3, channels[0], 7, stride=2, padding=3, IN=IN)
        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)
        self.conv2 = self._make_layer(
            blocks[0],
            layers[0],
            channels[0],
            channels[1],
            reduce_spatial_size=True,
            IN=IN
        )
        self.conv3 = self._make_layer(
            blocks[1],
            layers[1],
            channels[1],
            channels[2],
            reduce_spatial_size=True
        )
        self.conv4 = self._make_layer(
            blocks[2],
            layers[2],
            channels[2],
            channels[3],
            reduce_spatial_size=False
        )
        self.conv5 = Conv1x1(channels[3], channels[3])
        self.global_avgpool = nn.AdaptiveAvgPool2d(1)
        # fully connected layer
        self.fc = self._construct_fc_layer(
            self.feature_dim, channels[3], dropout_p=None
        )
        # identity classification layer
        self.classifier = nn.Linear(self.feature_dim, num_classes)

        self._init_params()

    def _make_layer(
        self,
        block,
        layer,
        in_channels,
        out_channels,
        reduce_spatial_size,
        IN=False
    ):
        layers = []

        layers.append(block(in_channels, out_channels, IN=IN))
        for i in range(1, layer):
            layers.append(block(out_channels, out_channels, IN=IN))

        if reduce_spatial_size:
            layers.append(
                nn.Sequential(
                    Conv1x1(out_channels, out_channels),
                    nn.AvgPool2d(2, stride=2)
                )
            )

        return nn.Sequential(*layers)

    def _construct_fc_layer(self, fc_dims, input_dim, dropout_p=None):
        if fc_dims is None or fc_dims < 0:
            self.feature_dim = input_dim
            return None

        if isinstance(fc_dims, int):
            fc_dims = [fc_dims]

        layers = []
        for dim in fc_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.BatchNorm1d(dim))
            layers.append(nn.ReLU(inplace=True))
            if dropout_p is not None:
                layers.append(nn.Dropout(p=dropout_p))
            input_dim = dim

        self.feature_dim = fc_dims[-1]

        return nn.Sequential(*layers)

    def _init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu'
                )
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def featuremaps(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        return x

    def forward(self, x, return_featuremaps=False):
        x = self.featuremaps(x)
        if return_featuremaps:
            return x
        v = self.global_avgpool(x)
        v = v.view(v.size(0), -1)
        if self.fc is not None:
            v = self.fc(v)
        if not self.training:
            return v
        y = self.classifier(v)
        if self.loss == 'softmax':
            return y
        elif self.loss == 'triplet':
            return y, v
        else:
            raise KeyError("Unsupported loss: {}".format(self.loss))


def init_pretrained_weights(model, key=''):
    """Initializes model with pretrained weights.
    
    Layers that don't match with pretrained layers in name or size are kept unchanged.
    """
    import os
    import errno
    import gdown
    from collections import OrderedDict

    def _get_torch_home():
        ENV_TORCH_HOME = 'TORCH_HOME'
        ENV_XDG_CACHE_HOME = 'XDG_CACHE_HOME'
        DEFAULT_CACHE_DIR = '~/.cache'
        torch_home = os.path.expanduser(
            os.getenv(
                ENV_TORCH_HOME,
                os.path.join(
                    os.getenv(ENV_XDG_CACHE_HOME, DEFAULT_CACHE_DIR), 'torch'
                )
            )
        )
        return torch_home

    torch_home = _get_torch_home()
    model_dir = os.path.join(torch_home, 'checkpoints')
    try:
        os.makedirs(model_dir)
    except OSError as e:
        if e.errno == errno.EEXIST:
            # Directory already exists, ignore.
            pass
        else:
            # Unexpected OSError, re-raise.
            raise
    filename = key + '_imagenet.pth'
    cached_file = os.path.join(model_dir, filename)

    if not os.path.exists(cached_file):
      if key == 'osnet_x0_25_endocv':
        filename = key + '.pth'
        cached_file = os.path.join(model_dir, filename)
        gdown.download(pretrained_urls[key], cached_file, quiet=False)
      elif key == 'osnet_dcn_x0_5_endocv':
        filename = 'osnet_x0_5_imagenet.pth'
        cached_file = os.path.join(model_dir, filename)
        gdown.download(pretrained_urls['osnet_x0_5'], cached_file, quiet=False)
      else:
        gdown.download(pretrained_urls[key], cached_file, quiet=False)

    state_dict = torch.load(cached_file)
    model_dict = model.state_dict()
    new_state_dict = OrderedDict()
    matched_layers, discarded_layers = [], []

    for k, v in state_dict.items():
        if k.startswith('module.'):
            k = k[7:] # discard module.

        if k in model_dict and model_dict[k].size() == v.size():
            new_state_dict[k] = v
            matched_layers.append(k)
        else:
            discarded_layers.append(k)

    model_dict.update(new_state_dict)
    model.load_state_dict(model_dict)

    if len(matched_layers) == 0:
        warnings.warn(
            'The pretrained weights from "{}" cannot be loaded, '
            'please check the key names manually '
            '(** ignored and continue **)'.format(cached_file)
        )
    else:
        print(
            'Successfully loaded imagenet pretrained weights from "{}"'.
            format(cached_file)
        )
        if len(discarded_layers) > 0:
            print(
                '** The following layers are discarded '
                'due to unmatched keys or layer size: {}'.
                format(discarded_layers)
            )

# Định nghĩa OSNet với DCN
class OSNetWithDCN(OSNet):
    def __init__(
        self,
        num_classes,
        blocks,
        layers,
        channels,
        feature_dim=512,
        loss='softmax',
        IN=False,
        **kwargs
    ):
        super(OSNetWithDCN, self).__init__(
            num_classes, blocks, layers, channels, feature_dim, loss, IN, **kwargs
        )
        # Thay conv5 bằng DeformableOSBlock
        self.conv5 = DeformableOSBlock(channels[3], channels[3])
        self._init_params()

    def featuremaps(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)  # Dùng DCN ở đây
        return x


##########
# Instantiation
##########
def osnet_x1_0(num_classes=1000, pretrained=True, loss='softmax', **kwargs):
    # standard size (width x1.0)
    model = OSNet(
        num_classes,
        blocks=[OSBlock, OSBlock, OSBlock],
        layers=[2, 2, 2],
        channels=[64, 256, 384, 512],
        loss=loss,
        **kwargs
    )
    if pretrained:
        init_pretrained_weights(model, key='osnet_x1_0')
    return model


def osnet_x0_75(num_classes=1000, pretrained=True, loss='softmax', **kwargs):
    # medium size (width x0.75)
    model = OSNet(
        num_classes,
        blocks=[OSBlock, OSBlock, OSBlock],
        layers=[2, 2, 2],
        channels=[48, 192, 288, 384],
        loss=loss,
        **kwargs
    )
    if pretrained:
        init_pretrained_weights(model, key='osnet_x0_75')
    return model


def osnet_x0_5(num_classes=1000, pretrained=True, loss='softmax', **kwargs):
    # tiny size (width x0.5)
    model = OSNet(
        num_classes,
        blocks=[OSBlock, OSBlock, OSBlock],
        layers=[2, 2, 2],
        channels=[32, 128, 192, 256],
        loss=loss,
        **kwargs
    )
    if pretrained:
        init_pretrained_weights(model, key='osnet_x0_5')
    return model


def osnet_x0_25(num_classes=1000, pretrained=True, loss='softmax', **kwargs):
    # very tiny size (width x0.25)
    model = OSNet(
        num_classes,
        blocks=[OSBlock, OSBlock, OSBlock],
        layers=[2, 2, 2],
        channels=[16, 64, 96, 128],
        loss=loss,
        **kwargs
    )
    if pretrained:
        init_pretrained_weights(model, key='osnet_x0_25')
    return model


def osnet_x0_25_endocv(num_classes=1000, pretrained=True, loss='softmax', **kwargs):
    # very tiny size (width x0.25)
    model = OSNet(
        num_classes,
        blocks=[OSBlock, OSBlock, OSBlock],
        layers=[2, 2, 2],
        channels=[16, 64, 96, 128],
        loss=loss,
        **kwargs
    )
    if pretrained:
        init_pretrained_weights(model, key='osnet_x0_25_endocv')
    return model


def osnet_ibn_x1_0(
    num_classes=1000, pretrained=True, loss='softmax', **kwargs
):
    # standard size (width x1.0) + IBN layer
    # Ref: Pan et al. Two at Once: Enhancing Learning and Generalization Capacities via IBN-Net. ECCV, 2018.
    model = OSNet(
        num_classes,
        blocks=[OSBlock, OSBlock, OSBlock],
        layers=[2, 2, 2],
        channels=[64, 256, 384, 512],
        loss=loss,
        IN=True,
        **kwargs
    )
    if pretrained:
        init_pretrained_weights(model, key='osnet_ibn_x1_0')
    return model

def osnet_dcn_x0_5_endocv(num_classes=1000, pretrained=True, loss='softmax', freeze_backbone=False, **kwargs):
    model = OSNetWithDCN(
        num_classes,
        blocks=[OSBlock, OSBlock, OSBlock],
        layers=[2, 2, 2],
        channels=[32, 128, 192, 256],
        loss=loss,
        **kwargs
    )
    if pretrained:
        #init_pretrained_weights(model, key='osnet_dcn_x0_5_endocv')  # Dùng weights của osnet_x1_0
        init_pretrained_weights(model, key='osnet_x0_5')
        if freeze_backbone:
            # Đóng băng conv1, conv2, conv3, conv4 dddddd
            for name, param in model.named_parameters():
                if 'conv5' not in name and 'fc' not in name and 'classifier' not in name:
                    param.requires_grad = False
            print("Backbone (conv1-conv4) frozen. Training only conv5, fc, and classifier.")
    return model


def main():
    from torchsummary import summary
    import torch

    # Giả sử bạn đã định nghĩa osnet_dcn_x0_5_endocv ở đây
    model = osnet_dcn_x0_5_endocv(num_classes=1000, pretrained=False)

    # Chuyển sang GPU nếu có
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Hiển thị cấu trúc với đầu vào 3x256x128 (thường dùng cho OSNet)
    summary(model, input_size=(3, 256, 128))
    
    
if __name__ == "__main__":
    main()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 32, 128, 64]           4,704
       BatchNorm2d-2          [-1, 32, 128, 64]              64
              ReLU-3          [-1, 32, 128, 64]               0
         ConvLayer-4          [-1, 32, 128, 64]               0
         MaxPool2d-5           [-1, 32, 64, 32]               0
            Conv2d-6           [-1, 32, 64, 32]           1,024
       BatchNorm2d-7           [-1, 32, 64, 32]              64
              ReLU-8           [-1, 32, 64, 32]               0
           Conv1x1-9           [-1, 32, 64, 32]               0
           Conv2d-10           [-1, 32, 64, 32]           1,024
           Conv2d-11           [-1, 32, 64, 32]             288
      BatchNorm2d-12           [-1, 32, 64, 32]              64
             ReLU-13           [-1, 32, 64, 32]               0
     LightConv3x3-14           [-1, 32,